In [1]:
import pandas as pd
file_path = "Data/Twitter_data/twitter_training.csv"
# FILEPATH: /c:/Users/Kaustubh/Downloads/Data_preproc_twitter.ipynb


# Read the CSV data
data = pd.read_csv(file_path)

In [2]:
import torch


# Create X and Y as 'Review' and 'Sentiment' respectively
X_text = data['Review'].values
Y_sent = data['Sentiment'].values
Y_label = []
# Initialize the LabelEncoder
Y_dict = {}
for y in Y_sent:
    if y not in Y_dict:
        Y_dict[y] = len(Y_dict)
    Y_label.append(Y_dict[y])
Y_tensor = torch.tensor(Y_label)
X_list = []
for x in X_text:
    X_list.append(str(x))



In [3]:
from sentence_transformers import SentenceTransformer
encoder_model = SentenceTransformer("all-MiniLM-L6-v2")
X = encoder_model.encode(X_list, convert_to_tensor=True)


/home/surajracha/miniconda3/envs/chatllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_tensor, test_size=0.2, random_state=42)

# Print the shapes of the train and test sets
print("Train set shape:", X_train.shape, Y_train.shape)
print("Test set shape:", X_test.shape, Y_test.shape)


Train set shape: torch.Size([59745, 384]) torch.Size([59745])
Test set shape: torch.Size([14937, 384]) torch.Size([14937])


In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [6]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_tensor, test_size=0.2, random_state=42)

# Print the shapes of the train and test sets
print("Train set shape:", X_train.shape, Y_train.shape)
print("Test set shape:", X_test.shape, Y_test.shape)

# Check if CUDA is available


# Move the tensors to the GPU


import torch.nn as nn
import torch.optim as optim

# Define the network architecture
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Hyperparameters
input_size = X.shape[1]  # Number of input features
hidden_size = 500  # Number of neurons in the hidden layer
num_classes = len(Y_dict)  # Number of output classes
num_epochs = 1000
learning_rate = 0.001

# Create the network
model = SimpleNN(input_size, hidden_size, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

# Convert Y_tensor to long ty
# Update the model and criterion to use GPU
model = model.to(device)
criterion = criterion.to(device)
X_train = X_train.to(device)
Y_train = Y_train.to(device)
# Train the model
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train).to(device)
    loss = criterion(outputs, Y_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if(epoch%100==0):
        

        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


Train set shape: torch.Size([59745, 384]) torch.Size([59745])
Test set shape: torch.Size([14937, 384]) torch.Size([14937])
Epoch [1/1000], Loss: 1.3869


Epoch [101/1000], Loss: 0.9766
Epoch [201/1000], Loss: 0.8432
Epoch [301/1000], Loss: 0.6415
Epoch [401/1000], Loss: 0.4331
Epoch [501/1000], Loss: 0.2857
Epoch [601/1000], Loss: 0.1981
Epoch [701/1000], Loss: 0.1481
Epoch [801/1000], Loss: 0.1193
Epoch [901/1000], Loss: 0.1025


In [7]:
# Predict the labels
# Predict the labels for the test set
with torch.no_grad():
    test_outputs = model(X_test.to(device))
    _, test_predicted = torch.max(test_outputs, 1)

# Calculate the accuracy on the test set
test_correct = (test_predicted == Y_test.to(device)).sum().item()
test_total = Y_test.size(0)
test_accuracy = test_correct / test_total

print('Accuracy of the model on the test set: {}%'.format(test_accuracy * 100))

_, predicted = torch.max(outputs, 1)



Accuracy of the model on the test set: 87.32007765950326%


In [24]:
device = torch.device("cpu")

In [25]:
class Net(nn.Module):
    def __init__(self,p):
        super(Net, self ).__init__()
        
        self.loss_reg = 0
        self.p =p 
        self.x = 0
        self.y = 0
        self.H_net1 = nn.Sequential(
            nn.Linear(384, 128),
            nn.Sigmoid(),
            nn.Linear(128, 256),
            nn.Sigmoid(),
            nn.Linear(256, 384*384).to(device)
        )
        self.X_net = nn.Sequential(
            nn.Linear(384, 512),
            nn.ReLU(),
            # nn.Linear(64, 128),
            # nn.ReLU(),
            # nn.Linear(128, 128),
            # nn.ReLU(),
            nn.Linear(512, 4),
            nn.Softmax(dim=2)

        )
        
    def forward(self, x):
        def H_mul(z):
            H12 = self.H_net1(z)
            H12= H12.reshape(z.shape[0],d,d)
            x12 = torch.matmul(z,H12)
            return(x12)
    
        
        def batch_jacobian(func, z, create_graph=False):
            # x in shape (Batch, Length)
            def _func_sum(z):
                return func(z).sum(dim=0)
            return torch.squeeze(torch.autograd.functional.jacobian(_func_sum, z, create_graph=create_graph)).permute(1,0,2)
        
        x.requires_grad =True
        p = self.p
        self.x = x
        d = x.shape[1]
        bs = x.shape[0]
        x= torch.unsqueeze(x,1)
        z = x.to(device)
        loss_reg = torch.zeros(bs,d).to(device)
        for i in range(p):
            H = self.H_net1(z).to(device)
            H = H.reshape(bs,d,d)
            z = torch.matmul(z,H).to(device)
            J = batch_jacobian(H_mul, z, create_graph=True)
            J_int =-torch.log(torch.abs(torch.det(J)))
            loss_reg = loss_reg + torch.squeeze(torch.autograd.grad(J_int, x,torch.ones_like(J_int),allow_unused=True,create_graph= True)[0]).to(device)
        self.loss_reg = loss_reg
        self.y = z
        y = self.X_net(z)
        return y


In [26]:
def train_model_priv(net,trainloader,optimizer,epochs,h,rate=10,device= torch.device('cuda'),print_cond = True,only_reg_flag=0,lr_schedular =None,lambda_loss=1):
    # scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
    print("Training")
    lr = lr_schedular
    # net = net.to(device)
    
    
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):  # loop over the dataset multiple times
        
        # scheduler.step()
        running_loss = 0.0
        running_loss_reg = 0.0
        if(lr):
       
            for groups in optimizer.param_groups:
                groups['lr'] = lr(epoch)
        # optimizer.param_groups[0]['lr'] = lr(epoch)
        
        
        for i, data in enumerate(trainloader, 0):
            print("Starter Loop: ",i)
            # get the inputs; data is a list of [inputs, labels]
            bs = len(data[0])
            
            inputs = data[0].to(device)
            inputs.requires_grad = True
            labels = data[1].to(device)
            print(inputs.shape,labels.shape)
            f = py_kde(inputs,inputs,h)
            f_der = py_kde_der(f,inputs)
            print(f.shape,f_der.shape)

            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = net(inputs)
            print(outputs.shape)
            if(only_reg_flag==1):
                loss = torch.norm(f_der/f.view(f.shape[0],1)+ net.loss_reg,dim=1).sum()
            elif(only_reg_flag==2):
                loss = criterion(torch.squeeze(outputs),torch.squeeze(labels))
                
            else:
                loss = lambda_loss*bs*criterion(torch.squeeze(outputs),torch.squeeze(labels)) + torch.norm(f_der/f.view(f.shape[0],1)+ net.loss_reg,dim=1).sum()
            loss.backward(retain_graph=True)

            optimizer.step()
            loss = loss.detach().cpu()/len(inputs)

          
            if(epoch ==0 and i==0):
                continue
      
            # loss_reg = torch.norm(f_der/f.view(f.shape[0],1)+ net.loss_reg,dim=1).sum().detach().cpu()/len(inputs)
            # wandb.log({"loss": loss.item(),"loss_reg":loss_reg.item()})

            # print statistics
            # print(loss.sum().shape)
            running_loss += loss.item()
            running_loss_reg += torch.norm(f_der/f.view(f.shape[0],1)+ net.loss_reg,dim=1).sum().item()
           # print every 2000 mini-batches
            if((i+1)%rate==0):

                if(print_cond):
                    
                    print("Epoch: ",epoch + 1,"Loss: " ,running_loss /(rate*trainloader.batch_size),"Reg Loss: ",running_loss_reg /(rate*trainloader.batch_size))
                    running_loss = 0.0
                    running_loss_reg = 0.0

In [27]:
def gau_ker(u):
    return torch.pow(2*torch.tensor(torch.pi),u.shape[1]/(-2))*torch.exp(torch.bmm(u.view(u.shape[0], 1, u.shape[1]), u.view(u.shape[0],  u.shape[1],1))/(-2)).to(device)

def py_kde(x,X_t,h):
    norm = X_t.shape[0]*(h**x.shape[1])
    prob = torch.zeros(x.shape[0]).to(device)
    for i in range(len(X_t)):
        prob+= (torch.squeeze(gau_ker((x - X_t[i])/h))/norm).to(device)
    return(prob)


def py_kde_der(p_x,x):
    # x.requires_grad = True
    # p_x = py_kde(x,X_t,h)
    return (torch.autograd.grad(p_x,x,torch.ones_like(p_x),allow_unused=True,create_graph=True)[0]).to(device)


def gau_ker_der(X,h):
    N= X.shape[0]
    d = X.shape[1]
    grad = torch.zeros(X.shape)
    for n in range(N):
        for i in range(d):
            for j in range(N):
                grad[n][i]+= torch.exp(-1*torch.dot((X[n]-X[j]),(X[n]-X[j]))/(2*h*h))*(X[n][i] -X[j][i]) /(N*(h**(d+2))*((2*math.pi)**(d/2)))

    return grad

In [28]:
batch_size_priv = 64
net_depth = 1

# Move the tensors to the GPU
X_train = X_train.to(device)
X_test = X_test.to(device)
Y_train = Y_train.to(device)
Y_test = Y_test.to(device)
train_priv = torch.utils.data.TensorDataset(X_train, Y_train)
test_priv = torch.utils.data.TensorDataset(X_test, Y_test)

trainloader_priv = torch.utils.data.DataLoader(train_priv, batch_size=batch_size_priv,
                                        shuffle=True, num_workers=2, drop_last=True)
testloader_priv = torch.utils.data.DataLoader(test_priv, batch_size=batch_size_priv,
                                        shuffle=False, num_workers=2, drop_last=True)


net = Net(net_depth)
print(sum(p.numel() for p in net.X_net.parameters() if p.requires_grad))

optim = torch.optim.Adam(net.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# wandb.watch(net)
train_flag = 0
lr_schedule = None


train_model_priv(net, trainloader_priv, optim, num_epochs, h=0.82, rate=1, device=device, only_reg_flag=train_flag, lr_schedular=lr_schedule,lambda_loss=1)

199172


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


torch.Size([64, 384]) torch.Size([64])
torch.Size([64]) torch.Size([64, 384])


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
